<a href="https://colab.research.google.com/github/padiketeku/Earth-Observation-Data-Programming/blob/main/Module4_SVM_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Acknowledgement

Google Earth Engine developers

# What is SVM classification?

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection. For classification tasks, the SVM algoithm would find the optimal hyperplane that maximises the margin between different classes.

Pros:

1, effective in high dimensional spaces.

2, effective in cases where number of dimensions is greater than the number of samples.

3, memmory efficient in that it uses a subset of training points in the decision function

4, versatile in that different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.


Cons:

1, if the number of features overly exceeds the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

2, SVMs do not directly provide probability estimates, these are calculated using expensive cross-validation methods


## Set working directory

In [4]:
#mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [5]:
#list of contents in the data folder
!ls /content/gdrive/MyDrive/data

ACT		     inflammation-04.csv  inflammation-10.csv  sentinel2
dem		     inflammation-05.csv  inflammation-11.csv  small-01.csv
gadm		     inflammation-06.csv  inflammation-12.csv  small-02.csv
inflammation-01.csv  inflammation-07.csv  inflammation.png     small-03.csv
inflammation-02.csv  inflammation-08.csv  machineLearning
inflammation-03.csv  inflammation-09.csv  osm


In [6]:
# change the directory to the appropriate working directory
import os
os.chdir('/content/gdrive/MyDrive/data/machineLearning')

In [7]:
# check to be sure of your working directory
!pwd

/content/gdrive/MyDrive/data/machineLearning


In [8]:
# set the path for results
#result_path = ' /content/gdrive/MyDrive/data/machineLearning/resultsFolder'

## Install/Import libraries

In [9]:
!pip install pycrs
import pycrs

# import required pandas and geopandas
import pandas as pd
import geopandas as gpd

# import earth engine
import ee

# import geemap
import geemap

# allow images to display in the notebook
from IPython.display import Image

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32686 sha256=b410d8eed7b027a5fabb843992c525559fe677201d068f1d09695688680930b4
  Stored in directory: /root/.cache/pip/wheels/b5/4a/72/1ba05f57ddf2cc80ad21a26512097762561d646ff3ff85f729
Successfully built pycrs


## Authenticate Earth Engine

1.   You need your Earth Engine project ID, which you can obtain from Assets
2. The project ID is the argument for the ee.Initialize()
*italicized text*

In [10]:
# Trigger the authentication command.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-racrabbe3')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


## Set up objects to filter the image collection

In [11]:
# list of coordinates defining the study area
listCoordinates = [[[149.154, -35.171],
          [149.189, -35.195],
          [149.210, -35.173],
          [149.172, -35.147]]]

# area of interest as an ee.Geometry
aoi = ee.Geometry.Polygon(listCoordinates)

# start date of range to filter for
start_date = '2022-02-01'

# end date
end_date = '2022-02-28'

## Collect the Sentinel-2 image from Earth Engine and filter this

In [12]:
# get the Sentinel-2 surface reflectance (SR) collection, filter by study area and date
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")\
            .filterBounds(aoi)\
            .filterDate(start_date,end_date)

This is an image collection, which means that more than one image file is available to work with. You may want to know the exact number of images in the collect. To do this, use **size** method. The **getInfo()** methods would alse be used to retrieve the computed value from the server.

In [13]:
# print the number of images in the collection
print('Total number:', s2.size().getInfo())

Total number: 10


The collection has 10 images.

Also, it is ideal to know the spectral bands making up the image. Once you see the bands google search "Sentinel-2 bands" so you can explain each bands, noting relevant and less relevant bands for a typical remote sensing project.

In [14]:
# find the spectral bands
s2.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60',
 'MSK_CLASSI_OPAQUE',
 'MSK_CLASSI_CIRRUS',
 'MSK_CLASSI_SNOW_ICE']

Another information you may be curious about is the different granules or tiles and the acquisition dates. For instance, you would like to know the acquisition date for the first image or tile in the collection.

In [15]:
# acquisition date for the first image in the collection
s2.first().get('GRANULE_ID').getInfo()

'L2A_T55HFB_A025653_20220203T000234'

## Cloud cover in multispectral optical imagery

It is not unusual to see cloud cover or cloud shadow in optical satellite imagery. Ideally, you would want an image with zero cloud cover to be certain about the image quality. It makes sense to always inpsect the satellite imagery to be sure of the level of cloud contamination.



We did not filter the collection by cloud cover. Since cloud cover is a major issue in optical remote sensing, you may want to have an idea on the amount of cloud cover in the image. This helps you to gauge the quality of the image, and hence decide whether to use it straight away, discard it or mask the pixels affected by cloud cover before application.

In [16]:
# check the cloud cover of the first tile in the collection
s2.first().get('CLOUDY_PIXEL_OVER_LAND_PERCENTAGE').getInfo()

27.644891

### Visualisation of the imagery in the collection

You now know there are 10 images in the collection and the first image has 27.6% cloud cover. You may be wondering if the other images have similar, lower or higher percentage cloud cover. And you might even be more keen to know whether the cloud cover is affecting your study area. This is the time to visualise each image of the collection and assess cloud cover for each tile.  

In [17]:
# put the images in a list
s2_list = s2.toList(s2.size());

In [18]:
# set some parameters for the images
parameters = {
                'min': 100,
                'max': 1000,
                'dimensions': 800, # square size in pixels
                'bands': ['B4', 'B3', 'B2'] # bands to display (r,g,b)
             }

In [19]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(s2.size().getInfo()):

    # when was this image taken?
    date = ee.Image(s2_list.get(i)).get('GRANULE_ID').getInfo()

    # cloud cover
    cloud = ee.Image(s2_list.get(i)).get('CLOUDY_PIXEL_OVER_LAND_PERCENTAGE').getInfo()

    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)

    # display the image
    display(Image(url = ee.Image(s2_list.get(i)).getThumbUrl(parameters)))
    ?Image


    # data to list
    this_data = [i,date,cloud]

    # append the data
    data.append(this_data)




Image # 0 L2A_T55HFB_A025653_20220203T000234 Cloud cover: 27.644891


Image # 1 L2A_T55HGB_A025653_20220203T000234 Cloud cover: 84.204847


Image # 2 L2A_T55HFB_A034633_20220208T000239 Cloud cover: 18.037216


Image # 3 L2A_T55HGB_A034633_20220208T000239 Cloud cover: 46.590626


Image # 4 L2A_T55HFB_A025796_20220213T000234 Cloud cover: 28.606695


Image # 5 L2A_T55HGB_A025796_20220213T000234 Cloud cover: 60.426736


Image # 6 L2A_T55HFB_A034776_20220218T000241 Cloud cover: 0.000171


Image # 7 L2A_T55HGB_A034776_20220218T000241 Cloud cover: 0.396


Image # 8 L2A_T55HFB_A025939_20220223T000235 Cloud cover: 71.826726


Image # 9 L2A_T55HGB_A025939_20220223T000235 Cloud cover: 97.781682


**Question- which image (acquisition date) had the most cloud cover?**

## Make an attribute table

Aside from the images, you may want to create an attribute table for easy understanding of the data. We would create a table that shows the tiles and cloud cover for each tile or image in the collection.

In [20]:
# To make a table the pandas DataFrame is used
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

# display the data frame
df

,Image #,Date,Cloud Cover
0,0,L2A_T55HFB_A025653_20220203T000234,27.644891
1,1,L2A_T55HGB_A025653_20220203T000234,84.204847
2,2,L2A_T55HFB_A034633_20220208T000239,18.037216
3,3,L2A_T55HGB_A034633_20220208T000239,46.590626
4,4,L2A_T55HFB_A025796_20220213T000234,28.606695
5,5,L2A_T55HGB_A025796_20220213T000234,60.426736
6,6,L2A_T55HFB_A034776_20220218T000241,0.000171
7,7,L2A_T55HGB_A034776_20220218T000241,0.396000
8,8,L2A_T55HFB_A025939_20220223T000235,71.826726
9,9,L2A_T55HGB_A025939_20220223T000235,97.781682


At the point, you should be certain that the **Image # 6 L2A_T55HFB_A034776_20220218T000241 Cloud cover: 0.000171** is the tile with the lowest cloud cover.

## Image visualisation using the geemap

Here, we will firstly select the image with the lowest cloud cover and display this using the geemap

In [21]:
#filter by cloud cover
filtered = s2.filter(ee.Filter.lte('CLOUDY_PIXEL_OVER_LAND_PERCENTAGE',10))
filtered

In [22]:
# call the geemap.Map to set up visualisation
m = geemap.Map()


#set the coordinates of your study area; you may include the zoom level
m.set_center(149.189, -35.195, 11)

In [23]:
#visualise the image with the lowest cloud cover
m.add_layer(
    filtered.first(),
    {'bands': ['B4', 'B3', 'B2'], 'min': 100, 'max': 1000},
    'image',
)


#visualise the study area polygon
m.add_layer(aoi,
    {},
    'Study area',)
#display the result
m

Map(center=[-35.195, 149.189], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

## Resize imagery

An image size can be reduced, via selecting the relevant spectral bands and limiting the dimension to the extent of the study area. This image has 26 bands, but only nine bands are relevant for the project purpose. The image size is larger than the study area, so this needs to be matched.

In [24]:
# select the best image
filtered = filtered.first()


#select relevant spectral bands
select_bands = ["B2", "B3", "B4","B5","B6","B7","B8","B11","B12"]

filtered = filtered.select(select_bands)

In [25]:
# clip the image to the study area
image2classify = filtered.clip(aoi)

#see the result
image2classify

In [26]:
#visualise the image with the lowest cloud cover
m.add_layer(
    image2classify,
    {'bands': ['B4', 'B3', 'B2'], 'min': 100, 'max': 1000},
    'Sentinel-2 image to classify',
)

#display layer
m

Map(center=[-35.195, 149.189], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [27]:
referenceData_path = ' /content/gdrive/MyDrive/data/machineLearning/ENV527-ML-Prac-actGrassland.shp'

## Load reference (vector data) using GeoPandas

In [28]:
#load the shapefile into a GeoDataFrame
referenceData = gpd.read_file(referenceData_path)

# Display the first few rows of the GeoDataFrame
print(referenceData .head())

   landcover                     geometry
0          0  POINT (149.17194 -35.15296)
1          0  POINT (149.16903 -35.15451)
2          0  POINT (149.18113 -35.15528)
3          0  POINT (149.17606 -35.15388)
4          0   POINT (149.1904 -35.16335)


In [29]:
# what is the CRS of this data?
referenceData.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Convert shapefile to feature collection object

In [30]:
 feature_collection = geemap.shp_to_ee('ENV527-ML-Prac-actGrassland.shp')

 feature_collection.first().geometry().projection().getInfo()

{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}

### Make a two-class problem

The high-quality native grass (class 1) and low-quality native grass (class 2) samples were merged to form one class. Thus, the classes are now 'exotic' and 'native' grasses.

In [31]:
# merge two classes into one
feature_collection = feature_collection.remap([0,1,2], [0,1,1], 'landcover')
feature_collection

In [48]:
#feature_collection = feature_collection.map(lambda feature.transform('EPSG:32755'))

## Collecting training areas

Here, the spectral values are attached to each reference point.

In [58]:
sample = image2classify.sampleRegions(

    #reference data
    collection=feature_collection,

    #keep this list of properties from the reference data
    properties=['landcover'],

    #match the CRS
    projection= 'EPSG:32755',

    #set the scale to get Sentinel-2 pixels
    scale=20,
)

sample

### Partition the reference data

The reference data are split into training and validation sets.

In [59]:
# The randomColumn() method will add a column of uniform random
# numbers in a column named 'random' by default.
sample = sample.randomColumn(seed=155)

split = 0.8  # Roughly 80% training, 20% testing

#training set
training = sample.filter(ee.Filter.lt('random', split))

# how many data points for training
display('Training size:', training.size())

#validation set
validation = sample.filter(ee.Filter.gte('random', split))

# how many data points for training
display('Validation size:', validation.size())

'Training size:'

'Validation size:'

## SVM classification

In [69]:
# Create an SVM classifier with custom parameters.
classifier = ee.Classifier.libsvm(**{
  'kernelType': 'LINEAR',
  'cost': 5
})

In [70]:
# Train the classifier.
trained = classifier.train(training, 'landcover', select_bands)

#detail about the trained variable
trained.explain()

In [71]:
# Classify the image.
classified = image2classify.classify(trained)

In [72]:
#visualise the image with the lowest cloud cover
m.add_layer(
    classified,
    {'palette': ['red', 'green'], 'min': 0, 'max': 1},
    'SVM:Classified Sentinel-2 image ',
)

#display layer
m

Map(bottom=79329.0, center=[-34.71565349880316, 148.1320953369141], controls=(WidgetControl(options=['position…

### Accuracy assessment

In [73]:
### Get a confusion matrix representing resubstitution accuracy.
train_accuracy = trained.confusionMatrix()

#display the error matrox table
display('Resubstitution error matrix:', train_accuracy)

#display the overall accuracy over the training data
display('Training overall accuracy:', train_accuracy.accuracy())

'Resubstitution error matrix:'

'Training overall accuracy:'

In [76]:
# Classify the validation data
validated = validation.classify(trained)


# Get a confusion matrix representing expected accuracy.
test_accuracy = validated.errorMatrix('landcover', 'classification')


#display the error matrix for test data
display('Validation error matrix:', test_accuracy)


#display the overall accuracy
display('Validation overall accuracy:', test_accuracy.accuracy())

'Validation error matrix:'

'Validation overall accuracy:'

#### End of Session

In [68]:
print( "End of the Session")

End of the Session
